# 原理

帧差法：

将连续两帧的图像数据进行差分法，即进行相减操作, 如果其相减后的绝对值大于阈值，则像素点变为255， 否则变为0，通过这种方法来找出视频中运动的物体


混合高斯模型：

将图像分为3-5个高斯模型，一个像素点来了，如果该像素点离任何一个高斯模型的距离大于其2倍的标准差，则为前景即运动物体，否则则是背景

           帧差法步骤：第一步：初始各种参数

           第二步：使用T帧图像构造模型，对于第一个帧图像的第一个像素点，使用u1，σ1构造高斯模型

           第三步：对于一个新来的模型，如果该像素在高斯模型3*σ1内，则属于该高斯模型，对参数进行更新

           第四步：如果不满足该高斯模型，重新建立一个新的高斯模型

# 步骤说明：

第一步：使用cv2.VideoCapture() 构造读取模型

第二步：使用cv2.getStructureElement(cv2.MORPH_ELLIPSE, （3, 3)) # 构造形态学使用的kernel，即np.ones((3, 3), np.uint8)

第三步：构造cv2.createBackgroundSubtractorMOG2() 实例化混合高斯模型

第四步：cap.read()从视频中读取文件，并使用model.apply(frame) 使用上混合高斯模型

第五步：使用cv2.morpholyEx(image, cv2.MORPH_OPEN, kernel) 使用开运算进行噪音的去除

第六步：cv2.findCountours找出图片中的轮廓，对其进行循环

第七步：对于周长大于188的轮廓，使用cv2.boundingRect计算外接矩阵，使用cv2.rectangle画出外接矩阵，作为人

第八步：使用cv2.imshow()展示图片，使用cv2.waitkey(150) & 0xff == 27来延长放映的时间

# MOG2算法
也是高斯混合模型分离算法，是MOG的改进算法。它基于Z.Zivkovic发布的两篇论文，即2004年发布的“Improved adaptive Gausian mixture model for background subtraction”和2006年发布的“Efficient Adaptive Density Estimation per Image Pixel for the Task of Background Subtraction”中提出。

该算法的一个重要特征是 它为每个像素选择适当数量的高斯分布，它可以更好地适应不同场景的照明变化等。

In [1]:
import cv2
import imageio
import numpy

In [2]:

def bgMOG2(cap):
    # 构造形态学使用的kernel，即np.ones((3, 3), np.uint8)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)) 
    # fgbg1 = cv2.createBackgroundSubtractorMOG()
    fgbg2 = cv2.createBackgroundSubtractorMOG2()
    frame_list=[]
    mask_list=[]
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print('not found')
            break
        frame = cv2.resize(frame, (1280,960), interpolation = cv2.INTER_CUBIC)
    #     fgmask1 = fgbg1.apply(frame)

        # 运用高斯模型进行拟合，在两个标准差内设置为0，在两个标准差外设置为255
        fgmask2 = fgbg2.apply(frame)
    #     fgmask3 = fgbg3.apply(frame)

        fgmask4 = cv2.morphologyEx(fgmask2, cv2.MORPH_OPEN, kernel, iterations=1)

        contours = cv2.findContours(fgmask4, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]
        for c in contours:
            # 第七步：进行人的轮廓判断，使用周长，符合条件的画出外接矩阵的方格
            length = cv2.arcLength(c, True)

            if length > 200:
                (x, y, w, h) = cv2.boundingRect(c)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1)


        frame_list.append(fgmask4)
        mask_list.append(frame[:,:,(2,1,0)])
        cv2.imshow("frame", frame)
        cv2.imshow("bgMOG2", fgmask4)
        if cv2.waitKey(110) & 0xff == 27:
            break
    return frame_list, mask_list
        

# 使用KNN根据前景面积检测运动物体

也是高斯混合模型分离算法，是MOG的改进算法。它基于Z.Zivkovic发布的两篇论文，即2004年发布的“Improved adaptive Gausian mixture model for background subtraction”和2006年发布的“Efficient Adaptive Density Estimation per Image Pixel for the Task of Background Subtraction”中提出。

该算法的一个重要特征是 它为每个像素选择适当数量的高斯分布，它可以更好地适应不同场景的照明变化等。

In [27]:
# 使用KNN根据前景面积检测运动物体
def bgSubtractorKNN(cap):
    history = 20    # 训练帧数
 
    bs = cv2.createBackgroundSubtractorKNN(detectShadows=True)  # 背景减除器，设置阴影检测
    bs.setHistory(history)
 
    frames = 0
    frame_list=[]
    mask_list=[]
 
    while True:
        res, tmp_frame = cap.read()      
        
        if not res:
            break
        frame = cv2.resize(tmp_frame,(1280,720))
        
        fg_mask = bs.apply(frame)   # 获取 foreground mask
 
        if frames < history:
            frames += 1
            continue
 
        # 对原始帧进行膨胀去噪
        th = cv2.threshold(fg_mask.copy(), 244, 255, cv2.THRESH_BINARY)[1]
        th = cv2.erode(th, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2)), iterations=2)
        dilated = cv2.dilate(th, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2)), iterations=2)
        # 获取所有检测框
        img, contours, hier = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
 

#         # draw a bounding box arounded the detected barcode and display the image
#         cv.drawContours(image, [box], -1, (0, 255, 0), 3)

#         c = sorted(contours, key=cv.contourArea, reverse=True)[0]
#         rect = cv.minAreaRect(c)
#         box = np.int0(cv.boxPoints(rect))
            
        for c in contours:
            # 获取矩形框边界坐标
            x, y, w, h = cv2.boundingRect(c)
            # 计算矩形框的面积
            area = cv2.contourArea(c)
            if 800 < area:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
        frame_list.append(dilated)
        mask_list.append(frame[:,:,(2,1,0)])
        cv2.imshow("frame", frame)
#         cv2.imshow("bgKNN", dilated)
        if cv2.waitKey(100) & 0xff == 27:
            break
    return frame_list, mask_list

# MOG算法，

即高斯混合模型分离算法，全称Gaussian Mixture-based Background/Foreground Segmentation Algorithm。2001年，由P.KadewTraKuPong和R.Bowden在论文“An improved adaptive background mixture model for real-time tracking with shadow detection”中提出。

它使用一种通过K高斯分布的混合来对每个背景像素进行建模的方法(K = 3〜5）。


In [7]:
def bgMOG(cap):
        # 构造形态学使用的kernel，即np.ones((3, 3), np.uint8)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)) 
    history = 20
    frame_list=[]
    mask_list=[]
    fgbg2 = cv2.bgsegm.createBackgroundSubtractorMOG(history)
    fgbg2.setHistory(history)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print('not found')
            break
        frame = cv2.resize(frame, (400,400), interpolation = cv2.INTER_CUBIC)
    #     fgmask1 = fgbg1.apply(frame)

        # 运用高斯模型进行拟合，在两个标准差内设置为0，在两个标准差外设置为255
        fgmask2 = fgbg2.apply(frame)
    #     fgmask3 = fgbg3.apply(frame)          
                
                

        fgmask4 = cv2.morphologyEx(fgmask2, cv2.MORPH_OPEN, kernel, iterations=1)

        img, contours,hier = cv2.findContours(fgmask4, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for c in contours:
            # 第七步：进行人的轮廓判断，使用周长，符合条件的画出外接矩阵的方格
            length = cv2.arcLength(c, True)

            if 50000 > length > 150:
                (x, y, w, h) = cv2.boundingRect(c)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)


        frame_list.append(fgmask4)
        mask_list.append(frame[:,:,(2,1,0)])
        cv2.imshow("frame", frame)
        cv2.imshow("bgMOG", fgmask4)
        if cv2.waitKey(110) & 0xff == 27:
            break
    return frame_list, mask_list
        

# GMG

该算法结合统计背景图像估计和每像素贝叶斯分割。由 Andrew B. Godbehere, Akihiro Matsukawa, Ken Goldberg在2012年的文章“Visual Tracking of Human Visitors under Variable-Lighting Conditions for a Responsive Audio Art Installation”中提出。
该算法使用前几个（默认为120）帧进行后台建模。它采用概率前景分割算法，使用贝叶斯推理识别可能的前景对象。

In [5]:
def bgGMG(cap):
        # 构造形态学使用的kernel，即np.ones((3, 3), np.uint8)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)) 
    history = 20
    frame_list=[]
    mask_list=[]
    
    # fgbg1 = cv2.createBackgroundSubtractorMOG()
    fgbg2 = cv2.bgsegm.createBackgroundSubtractorGMG(initializationFrames= history)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print('not found')
            break
        frame = cv2.resize(frame, (400,400), interpolation = cv2.INTER_CUBIC)

        # 运用高斯模型进行拟合，在两个标准差内设置为0，在两个标准差外设置为255
        fgmask2 = fgbg2.apply(frame)

        fgmask2 = cv2.morphologyEx(fgmask2, cv2.MORPH_OPEN, kernel, iterations=1)

        contours = cv2.findContours(fgmask2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]
        for c in contours:
            # 第七步：进行人的轮廓判断，使用周长，符合条件的画出外接矩阵的方格
            length = cv2.arcLength(c, True)

            if 500 > length > 100:
                (x, y, w, h) = cv2.boundingRect(c)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1)


        frame_list.append(fgmask2)
        mask_list.append(frame[:,:,(2,1,0)])
        cv2.imshow("img", frame)
        cv2.imshow("bgGMG", fgmask2)
        if cv2.waitKey(110) & 0xff == 27:
            break
    return frame_list, mask_list
        

In [28]:
if __name__ == '__main__':
    cap = cv2.VideoCapture("/data/disk2/01_dataset/02_shuohuang/04_classification/01hand.mp4")
    bgKNN = "bgKNN.gif"
    bg_MOG = "bgMOG.gif"
    bg_MOG2 = "bgMOG2.gif"
    bg_GMG = "bgGMG.gif"
    
    oriKNN = "oriKNN.gif"
    oriMOG = "oriMOG.gif"
    
    oriMOG2 = "oriMOG2.gif"
    oriGMG = "oriGMG.gif"

    frame_list, mask_list = bgSubtractorKNN(cap)
#     frame_list, mask_list = bgMOG2(cap)
#     frame_list, mask_list = bgMOG(cap)
#     frame_list, mask_list = bgGMG(cap)
    
#     imageio.mimsave(bg_GMG, mask_list, 'GIF', duration=0.05)
#     imageio.mimsave(oriGMG, frame_list, 'GIF', duration=0.05)
    cap.release()
    cv2.destroyAllWindows()